In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 1

%load_ext watermark
%watermark -a 'stekaiser' -d -t -v -p numpy,pandas,matplotlib,keras,tensorflow -g

Using TensorFlow backend.


stekaiser 2016-11-30 11:20:06 

CPython 3.5.2
IPython 5.1.0

numpy 1.11.2
pandas 0.19.1
matplotlib 1.5.3
keras 1.1.1
tensorflow 0.11.0
Git hash: 6c25dea87295c3a00dd4327adf0f0666af6e890c


In [2]:
import os
import sys

project_dir = os.path.join(os.getcwd(), os.pardir)
sys.path.insert(0, project_dir)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import zipfile
from dotenv import find_dotenv, load_dotenv

# show all columns
pd.set_option('display.max_columns', None)

In [3]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Convolution1D
from keras.layers import MaxPooling1D,Flatten
from keras.layers import GRU
from keras.layers import Dropout
from keras.regularizers import l2, activity_l2
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import TensorBoard,ModelCheckpoint,EarlyStopping,Callback
# fix random seed for reproducibility
np.random.seed(7)

sys.setrecursionlimit(50000)

In [4]:
import itertools
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

In [5]:
batch_size = 64
nTimesteps = 128

### Load data
`(time x channels)`

In [6]:
X = pd.read_csv('https://dl.dropboxusercontent.com/u/558313/keras-js/test_new.csv', nrows=nTimesteps*batch_size, header=None)
X.head()

,0,1,2,3,4,5
0,0.620713,0.463101,0.869851,0.610158,0.702451,0.767094
1,0.797397,0.441206,0.753544,0.679309,0.524256,0.752844
2,0.587150,0.137027,0.938175,0.590607,0.840056,0.652926
3,0.176337,0.909013,0.356048,0.306722,0.134125,0.669478
4,0.790946,0.155954,0.173121,0.310723,0.989607,0.294101


In [7]:
X = X.as_matrix()

In [8]:
X.shape

(8192, 6)

Make it `(batchSize x nTimesteps, nChannels)`

In [9]:
X = np.reshape(X, (-1, nTimesteps, 6))
X.shape

(64, 128, 6)

In [10]:
target = np.hstack((np.ones((1,batch_size/2)), np.zeros((1,batch_size/2))))
target = np.transpose(target)

/home/steffen/bin/anaconda3/envs/equestic/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)
/home/steffen/bin/anaconda3/envs/equestic/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


In [11]:
Y = np.column_stack((target, np.flipud(target)))

In [12]:
Y.shape

(64, 2)

In [13]:
n_hidden = 64

model = Sequential()
model.add(LSTM(n_hidden,
               return_sequences=True,
               batch_input_shape=(batch_size, nTimesteps, X.shape[2])
              ))
model.add(Dropout(0.2))
model.add(LSTM(n_hidden))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [14]:
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (64, 128, 64)         18176       lstm_input_1[0][0]               
____________________________________________________________________________________________________
dropout_1 (Dropout)              (64, 128, 64)         0           lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (64, 64)              33024       dropout_1[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)              (64, 64)              0           lstm_2[0][0]                     
___________________________________________________________________________________________

In [15]:
history = model.fit(X,Y,batch_size=batch_size)

Epoch 1/10
64/64 [==============================] - 1s - loss: 0.7095 - acc: 0.4688
Epoch 2/10
64/64 [==============================] - 0s - loss: 0.7163 - acc: 0.5156
Epoch 3/10
64/64 [==============================] - 0s - loss: 0.8082 - acc: 0.5000
Epoch 4/10
64/64 [==============================] - 0s - loss: 0.7601 - acc: 0.5000
Epoch 5/10
64/64 [==============================] - 0s - loss: 0.7371 - acc: 0.4844
Epoch 6/10
64/64 [==============================] - 0s - loss: 0.6814 - acc: 0.5781
Epoch 7/10
64/64 [==============================] - 0s - loss: 0.6991 - acc: 0.4844
Epoch 8/10
64/64 [==============================] - 0s - loss: 0.7240 - acc: 0.4062
Epoch 9/10
64/64 [==============================] - 0s - loss: 0.6970 - acc: 0.5312
Epoch 10/10
64/64 [==============================] - 0s - loss: 0.6780 - acc: 0.6094


In [16]:
model.save_weights('model.hdf5')
with open('model.json', 'w') as f:
    f.write(model.to_json())

In [17]:
!python encoder.py model.hdf5

In [18]:
predictions = model.predict(X, batch_size=batch_size)
predictions

array([[ 0.5048058 ,  0.49519429],
       [ 0.48626968,  0.51373035],
       [ 0.51306689,  0.48693311],
       [ 0.5060125 ,  0.49398756],
       [ 0.50179213,  0.49820781],
       [ 0.51991212,  0.48008794],
       [ 0.52513933,  0.47486073],
       [ 0.49827269,  0.50172734],
       [ 0.49479485,  0.50520521],
       [ 0.50521934,  0.49478063],
       [ 0.49181676,  0.5081833 ],
       [ 0.51152986,  0.48847014],
       [ 0.51833814,  0.48166192],
       [ 0.49721789,  0.50278205],
       [ 0.49942103,  0.500579  ],
       [ 0.50356525,  0.49643466],
       [ 0.5063166 ,  0.49368343],
       [ 0.52224225,  0.47775775],
       [ 0.50790608,  0.49209398],
       [ 0.51393336,  0.48606667],
       [ 0.50063878,  0.49936128],
       [ 0.50423473,  0.49576533],
       [ 0.50791854,  0.49208149],
       [ 0.50661182,  0.49338815],
       [ 0.5019998 ,  0.4980002 ],
       [ 0.49754021,  0.50245982],
       [ 0.51188141,  0.48811862],
       [ 0.52017665,  0.47982338],
       [ 0.49647522,